# Multi-Table Synthesis - Advanced Usage

### How to synthesize data from a database schema

Relational databases (RDBMS) are a type of data storage that allows users to access data that is stored in various tables connected through primary and foreign keys. They serve a variety of different use cases, as they offer benefits such as security and reliability. 

For many data science scenarios, a single-table model is usually the go-to, but the truth is that RDBMS and table-like storages are important for more complex use cases, such as systems testing, replicating a database for customer segmentation, or even for data migrations between on-prem and the cloud. 

YData Fabric offers an easy-to-use and familiar interface through the SDK to support Multi-Table Synthesis. With the SDK and a few lines of code, users can replicate full relational databases while maintaining the consistency of all the keys and the statistical information of cross-table relations. 

[Berka](https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions) is the dataset chosen to demo Fabric Multi-Table synthesis properties and interface. 

## Getting the data from an RDBMS

We can load an RDBMS connector created through the Fabric platform using the code below.

In [1]:
from ydata.labs import Connectors
connector = Connectors.get(uid='<uid>', namespace='<namespace>').connector

Alternatively, the RDBMS connector can be created through the SDK.

In [ ]:
from ydata.connectors.storages.rdbms_connector import MySQLConnector

USERNAME = '<username>'
PASSWORD = '<password>'
HOSTNAME = '<hostname>'
PORT = '3306'
DATABASE_NAME = '<database>'

conn_str = {
    "hostname": f'{HOSTNAME}',
    "username": f'{USERNAME}',
    "password": f'{PASSWORD}',
    "port": f'{PORT}',
    "database": f'{DATABASE_NAME}'
}

connector = MySQLConnector(conn_string=conn_str)

The entire database can be read with the `read_database` method. 

In [2]:
data = connector.read_database()

We can also access the entire database by creating a `MultiDataset` that receives the connector.

In [3]:
from ydata.dataset.multidataset import MultiDataset

data = MultiDataset(connector)

Alternatively, it is possible to read only specific tables using the `get_tables` method and passing the list of tables. In the following example, we retrieve the database schema and ask for all available tables.

In [4]:
schema = connector.get_database_schema()
tables = list(schema.tables.keys())
data = connector.get_tables(tables)

We can now display the data of the `loan` table.

In [5]:
data['loan'].to_pandas()

,loan_id,account_id,date,amount,duration,payments,status,interest_rate,interest_amount
0,4959,2,940105,80952,24,3373.0,A,0.02,1619.04
1,4961,19,960429,30276,12,2523.0,B,0.04,1211.04
2,4962,25,971208,30276,12,2523.0,A,0.05,1513.80
3,4967,37,981014,318480,60,5308.0,D,0.01,3184.80
4,4968,38,980419,110736,48,2307.0,C,0.08,8858.88
...,...,...,...,...,...,...,...,...,...
677,7294,11327,980927,39168,24,1632.0,C,0.08,3133.44
678,7295,11328,980718,280440,60,4674.0,C,0.09,25239.60
679,7304,11349,951029,419880,60,6998.0,C,0.02,8397.60
680,7305,11359,960806,54024,12,4502.0,A,0.02,1080.48


### Lazy and eager loading

The `MultiDataset` is by default lazy loaded. No tables are initially fetched.

In [6]:
data = connector.read_database()
print(data)

MultiDataset Summary 
 
Number of tables: 9 
 
  Table name Num cols Num rows Primary keys Foreign keys                                 Notes
0     append                                              This table has not been fetched yet.
1   district                                              This table has not been fetched yet.
2    account                                              This table has not been fetched yet.
3     client                                              This table has not been fetched yet.
4       disp                                              This table has not been fetched yet.
5       loan                                              This table has not been fetched yet.
6      order                                              This table has not been fetched yet.
7      trans                                              This table has not been fetched yet.
8       card                                              This table has not been fetched yet.


Each table is fetched the first time it is accessed. 

In [7]:
data["account"]
print(data)

MultiDataset Summary 
 
Number of tables: 9 
 
  Table name Num cols Num rows  Primary keys   Foreign keys                                 Notes
0     append                                                 This table has not been fetched yet.
1   district                                                 This table has not been fetched yet.
2    account        4     4500  [account_id]  [district_id]                                      
3     client                                                 This table has not been fetched yet.
4       disp                                                 This table has not been fetched yet.
5       loan                                                 This table has not been fetched yet.
6      order                                                 This table has not been fetched yet.
7      trans                                                 This table has not been fetched yet.
8       card                                                 This table

To fetch all tables at once we can call the `compute` method.

In [8]:
data.compute()
print(data)

/home/ydata/.venv/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 9.55 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


MultiDataset Summary 
 
Number of tables: 9 
 
  Table name  Num cols  Num rows  Primary keys             Foreign keys Notes
0     append         3        20                                             
1   district        16        77          [a1]                               
2    account         4      4500  [account_id]            [district_id]      
3     client         6      5369   [client_id]            [district_id]      
4       disp         4      5369     [disp_id]  [client_id, account_id]      
5       loan         9       682     [loan_id]             [account_id]      
6      order         6      6471    [order_id]             [account_id]      
7      trans        10    135000    [trans_id]             [account_id]      
8       card         4       892     [card_id]                [disp_id]      


Alternatively, we can disable the lazy loading when calling the connector methods or when creating a `MultiDataset` by setting the `lazy` flag to `false`.

In [9]:
data = connector.read_database(lazy=False)
print(data)

/home/ydata/.venv/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 9.55 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


MultiDataset Summary 
 
Number of tables: 9 
 
  Table name  Num cols  Num rows  Primary keys             Foreign keys Notes
0     append         3        20                                             
1   district        16        77          [a1]                               
2    account         4      4500  [account_id]            [district_id]      
3     client         6      5369   [client_id]            [district_id]      
4       disp         4      5369     [disp_id]  [client_id, account_id]      
5       loan         9       682     [loan_id]             [account_id]      
6      order         6      6471    [order_id]             [account_id]      
7      trans        10    135000    [trans_id]             [account_id]      
8       card         4       892     [card_id]                [disp_id]      


## Synthesizer definition, training, and sampling

### Setting the Metadata parameters

In a relational database schema, some tables might refer to time-series datasets but others might just hold data attributes such as customers' information.

In this example, the `Berka` database transactions table can be considered a time series. For that reason, the table **trans** needs to be set as a `timeseries` and the column `date` as the table time order reference (**sortbykey**).

In [10]:
data = connector.read_database()

In [11]:
from ydata.metadata.multimetadata import MultiMetadata

dataset_type = {
    'trans': 'timeseries'
}

dataset_attrs = {
    'trans': {
        'sortbykey': 'date',
        'entities': []
    }
}

m = MultiMetadata(data, dataset_attrs=dataset_attrs, dataset_type=dataset_type)

The `MultiMetadata` object is lazy loaded when the `MultiDataset` object also is.

In [12]:
print(m)

MultiMetadata Summary 
 
Number of tables: 9 
 
  Table name # cols Primary keys Foreign keys PK characteristics FK characteristics                                                    Notes
0     append                                                                         The Metadata for this table has not been requested yet.
1   district                                                                         The Metadata for this table has not been requested yet.
2    account                                                                         The Metadata for this table has not been requested yet.
3     client                                                                         The Metadata for this table has not been requested yet.
4       disp                                                                         The Metadata for this table has not been requested yet.
5       loan                                                                         The Metadata for this

When we access the `Metadata` of a table for the first time, the data is automatically fetched. We can also force fetch all tables by calling the `compute` method.

In [13]:
m["account"]
print(m)

MultiMetadata Summary 
 
Number of tables: 9 
 
  Table name # cols Primary keys   Foreign keys PK characteristics       FK characteristics                                                    Notes
0     append                                                                                 The Metadata for this table has not been requested yet.
1   district                                                                                 The Metadata for this table has not been requested yet.
2    account      4   account_id  [district_id]               [id]  {'district_id': ['id']}                                                         
3     client                                                                                 The Metadata for this table has not been requested yet.
4       disp                                                                                 The Metadata for this table has not been requested yet.
5       loan                                              

In [14]:
m.compute()
print(m)

/home/ydata/.venv/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 9.55 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/ydata/.venv/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 9.66 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/ydata/.venv/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 9.56 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/ydata/.venv/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 9.55 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


MultiMetadata Summary 
 
Number of tables: 9 
 
  Table name  # cols Primary keys             Foreign keys PK characteristics                           FK characteristics Notes
0     append       3                                                                                                            
1   district      16           a1                                        [id]                                                   
2    account       4   account_id            [district_id]               [id]                      {'district_id': ['id']}      
3     client       6    client_id            [district_id]               [id]                      {'district_id': ['id']}      
4       disp       4      disp_id  [client_id, account_id]               [id]  {'client_id': ['id'], 'account_id': ['id']}      
5       loan       9      loan_id             [account_id]               [id]                       {'account_id': ['id']}      
6      order       6     order_id             [ac

### Setting the synthesizer configurations

In this example, we use two main functionalities available for the synthetic generation: the **anonymizer** and the **calculated features**.

The anonymizer allows the replacement of personally identifiable information for other non-identifiable data. This feature can be applied to any column, including primary and foreign keys.

In this example, the following columns are anonymized:
- The `district` table primary key (`district.a1`) and all its references (i.e., foreign keys) across the database, such as the `account.district_id`. The replacement data will be generated according to the specified regex expression.
- All the primary keys from the `account` table, as well as their references across the database. The replacement data are integers.
- The values from the `bank_to` column from the `order` table will be replaced by city names (other strategies are available according to the specified `AnonymizerType`).

In [15]:
from ydata.preprocessors.methods.anonymization import AnonymizerType

anonymizer_config = {
    'district': {
        'a1': r'[0-9]{4}-[A-Z]{5}'
    },
    'account': {
        'anonymize_primary_keys': True
    },
    'order': {
        'bank_to': AnonymizerType.CITY
    }
}

The calculated features functionality allows the generation of specific columns based on data from other columns according to the business rules specified in custom functions.

In this example, the following columns are calculated features:
- The `full_name` column from the `client` table is generated by concatenating the first and last names of each client, which are available in the `first_name` and `last_name` columns of the same table.
- The `a10_sum` column from the `client` table is generated by summing all the values from the `a10` column of the `district` table for each client. Since this is an inter-table calculated feature (i.e., several tables are used), there is a need to establish the relationship between the tables (in this case, between the `client` and the `district`). The user should include the primary and foreign keys in the base columns, and establish the relationship inside the custom function (see the `get_a10_sum` function).

In [16]:
import pandas as pd
import numpy as np

def get_full_name(first_name, last_name):
    full_names = []
    for ix in range(first_name.shape[0]):
        full_names.append(first_name[ix].strip() + " " + last_name[ix].strip())
    return np.asarray(full_names)

def get_a10_sum(client_id, district_id, a1, a10):
    a1_s = pd.Series(a1, name="a1")
    a10_s = pd.Series(a10, name="a10")
    district_data = pd.concat([a1_s, a10_s], axis=1)
    a10_sum = pd.Series(0, index=client_id)
    for c, d in zip(client_id, district_id):
        a10_sum[c] = district_data[district_data["a1"] == d]["a10"].sum()
    return a10_sum.values

calculated_features=[
    {
      "calculated_features": "client.full_name",
      "function": get_full_name,
      "calculated_from": ["client.first_name", "client.last_name"],
    },
    {
      "calculated_features": "client.a10_sum",
      "function": get_a10_sum,
      "calculated_from": ["client.client_id", "client.district_id", "district.a1", "district.a10"]
    }
]

### Training the synthesizer

We can now train the synthesizer by creating a `MultiTableSynthesizer` and passing the data, the metadata, and the configurations for the anonymizer and the calculated features. Depending on the size of the database, the training may take a while.

In [17]:
from ydata.synthesizers.multitable.model import MultiTableSynthesizer

synth = MultiTableSynthesizer()
synth.fit(data, m, anonymize=anonymizer_config, calculated_features=calculated_features)

INFO: 2023-12-29 12:07:49,237 (1/9) - Fitting table: [district]
INFO: 2023-12-29 12:07:50,644 [SYNTHESIZER] - Number columns considered for synth: 16
INFO: 2023-12-29 12:07:50,862 [SYNTHESIZER] - Starting the synthetic data modeling process over 1x1 blocks.
INFO: 2023-12-29 12:07:50,864 [SYNTHESIZER] - Preprocess segment
INFO: 2023-12-29 12:07:50,871 [SYNTHESIZER] - Synthesizer init.
INFO: 2023-12-29 12:07:50,872 [SYNTHESIZER] - Processing the data prior fitting the synthesizer.
INFO: 2023-12-29 12:07:51,031 (2/9) - Fitting table: [client]
INFO: 2023-12-29 12:07:53,221 [SYNTHESIZER] - Number columns considered for synth: 22
INFO: 2023-12-29 12:07:53,592 [SYNTHESIZER] - Starting the synthetic data modeling process over 1x1 blocks.
INFO: 2023-12-29 12:07:53,595 [SYNTHESIZER] - Preprocess segment
INFO: 2023-12-29 12:07:53,600 [SYNTHESIZER] - Synthesizer init.
INFO: 2023-12-29 12:07:53,601 [SYNTHESIZER] - Processing the data prior fitting the synthesizer.
INFO: 2023-12-29 12:07:54,548 (3/9

### Sampling

Since there is a need to keep the consistency of the tables, as well as the referential integrity, to sample from trained synthesizers the number of records is set through a ratio based on the original number of records (e.g., 1.0 is equivalent to the size of the original database).

In [18]:
sample = synth.sample(n_samples=1.)

INFO: 2023-12-29 12:08:44,223 (1/9) - Synthesizing table: district
INFO: 2023-12-29 12:08:44,224 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:08:44,850 (2/9) - Synthesizing table: client
INFO: 2023-12-29 12:08:45,047 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:08:45,833 (3/9) - Synthesizing table: disp
INFO: 2023-12-29 12:08:45,923 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:08:46,390 (4/9) - Synthesizing table: card
INFO: 2023-12-29 12:08:46,445 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:08:46,768 (5/9) - Synthesizing table: account
INFO: 2023-12-29 12:08:46,853 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:08:47,449 (6/9) - Synthesizing table: order
INFO: 2023-12-29 12:08:47,528 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:08:48,290 (7/9) - Synthesizing table: loan
INFO: 2023-12-29 12:08:48,414 [SYNTHESIZER] - Start generating model samples.
INF

We can now display the sampled data of the `client`, `district`, `account`, and `order` tables.

In [19]:
sample['client'].to_pandas()

,client_id,birth_number,district_id,first_name,last_name,full_name
0,1,310921,0621-RZFZF,Christopher,Green,Christopher Green
1,2,455922,0621-RZFZF,Eleanor,Rodriguez,Eleanor Rodriguez
2,3,795813,0621-RZFZF,Nicholas,Wilson,Nicholas Wilson
3,4,671030,0621-RZFZF,John,Garcia,John Garcia
4,5,310921,0621-RZFZF,Christopher,Rodriguez,Christopher Rodriguez
...,...,...,...,...,...,...
4708,4709,775929,7902-IIIGC,Justin,Young,Justin Young
4709,4710,395811,7902-IIIGC,Evelyn,Anderson,Evelyn Anderson
4710,4711,235505,7902-IIIGC,Hannah,Carter,Hannah Carter
4711,4712,605803,7902-IIIGC,Sophia,Phillips,Sophia Phillips


In [20]:
sample['district'].to_pandas()

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
0,0621-RZFZF,Jesenik,north Moravia,127369,32,19,13,2,8,80.0,8173,5.77,7.0,116,1580,6872
1,2156-JXQCO,Hradec Kralove,east Bohemia,88768,77,26,6,2,6,59.0,8507,1.67,3.0,90,1328,2325
2,2329-FXXJT,Brno - venkov,south Moravia,161954,21,41,10,3,11,75.0,8720,1.60,3.0,116,2854,3651
3,7093-QQUYU,Blansko,east Bohemia,161854,83,21,5,1,3,85.0,9538,1.72,9.0,125,2854,4743
4,2187-EUPQP,Usti nad Orlici,east Bohemia,78955,50,24,7,1,9,59.0,8173,1.89,3.0,124,1655,1525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,4499-ZYNRO,Havlickuv Brod,north Bohemia,118650,17,12,1,2,7,80.0,9832,6.63,1.0,87,6949,6872
73,6421-HXHHF,Semily,east Bohemia,162580,59,21,4,2,6,85.0,9538,1.02,3.0,94,2854,5410
74,9085-NEWAF,Brno - venkov,south Moravia,387570,0,0,0,1,1,100.0,9897,1.60,2.0,140,18721,18696
75,8753-KICBR,Mlada Boleslav,north Bohemia,112065,25,19,6,2,8,57.0,9065,2.80,4.0,124,4289,2618


In [21]:
sample['account'].to_pandas()

,account_id,district_id,frequency,date
0,09359461,2273-ODBBZ,POPLATEK MESICNE,930226
1,57748209,9354-FYKEC,POPLATEK MESICNE,930226
2,56990587,5246-BMJZN,POPLATEK MESICNE,970530
3,24404964,7329-BUHDH,POPLATEK MESICNE,930226
4,51850639,2842-QSIAJ,POPLATEK MESICNE,950921
...,...,...,...,...
3921,03203087,6473-FABQY,POPLATEK MESICNE,961005
3922,18220766,2994-TBMMM,POPLATEK MESICNE,961005
3923,49309729,5226-JLHXV,POPLATEK MESICNE,970708
3924,24420997,1072-NEWAC,POPLATEK MESICNE,930114


In [22]:
sample['order'].to_pandas()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,1,09359461,New Deanna,12663782.0,1877.0,
1,2,57748209,Jaybury,15017515.0,210.0,
2,3,56990587,West Kellyside,62572306.0,1065.0,
3,4,24404964,Jaybury,53641200.0,115.0,POJISTNE
4,5,51850639,West Kellyside,71683752.0,400.0,
...,...,...,...,...,...,...
6834,6835,03203087,Laurenbury,35869842.0,859.0,
6835,6836,18220766,South Elizabeth,48207306.0,164.0,
6836,6837,49309729,Laurenbury,35869842.0,1439.0,SIPO
6837,6838,24420997,New Deanna,27317208.0,2683.0,SIPO


### Validating the schema of the synthetic sample

As mentioned before, one of the goals of the multi-table synthesizer is to keep the consistency of the tables, which means that the schema of the synthetic sample should be the same as in the original database. In particular, the primary keys should be respected (including their uniqueness), and the referential integrity should be ensured.

The `MultiMetadata` offers the `get_schema_validation_summary` method, which returns a validation summary of the described aspects. We can see for the entire database and for each table which primary and foreign keys are violated. This information is also aggregated in a single relationship quality metric, bounded between 0 and 100%. Ideally, the relationship quality should be 100% and no primary keys should be violated.

In [23]:
m_sample = MultiMetadata(sample)
print(m_sample.get_schema_validation_summary(m, sample, data))

Schema Validation Summary

Number of Primary Key Violations: 0
Number of Foreign Key Violations: 0
Relationship Quality: 100%


Table append

	Primary Keys
		Current Schema: None
		Reference Schema: None

	Non-Matching Primary Keys: None

	Non-Matching Foreign Keys: None


Table district

	Primary Keys
		Current Schema: a1 [VALID]
		Reference Schema: a1 [VALID]

	Non-Matching Primary Keys: None

	Non-Matching Foreign Keys: None


Table account

	Primary Keys
		Current Schema: account_id [VALID]
		Reference Schema: account_id [VALID]

	Foreign Key 1
		Current Schema: district_id -> district.a1 (1-N)
		Reference Schema: district_id -> district.a1 (1-N)

	Percentage of Valid Foreign Keys
		Current Schema: 100%
		Reference Schema: 100%

	Non-Matching Primary Keys: None

	Non-Matching Foreign Keys: None


Table client

	Primary Keys
		Current Schema: client_id [VALID]
		Reference Schema: client_id [VALID]

	Foreign Key 1
		Current Schema: district_id -> district.a1 (1-N)
		Reference Schema:

### Persisting the synthetic sample

When sampling from the synthesizer there is the possibility of persisting the synthetic sample directly to a database. For that, we just have to specify a connector for the destination database and how the persist should behave if the tables already exist (`if_exists` parameter):
- fail: raise an InvalidTableException.
- replace: drop the table before inserting new values.
- append: insert new values into the existing table.

In [24]:
from ydata.labs import Connectors
synth_connector = Connectors.get(uid='<uid>', namespace='<namespace>').connector

In [25]:
sample = synth.sample(n_samples=1., connector=synth_connector, if_exists="append")

INFO: 2023-12-29 12:09:35,422 (1/9) - Synthesizing table: district
INFO: 2023-12-29 12:09:35,424 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:09:36,604 (2/9) - Synthesizing table: client
INFO: 2023-12-29 12:09:36,809 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:09:37,583 (3/9) - Synthesizing table: disp
INFO: 2023-12-29 12:09:37,675 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:09:38,166 (4/9) - Synthesizing table: card
INFO: 2023-12-29 12:09:38,220 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:09:38,778 (5/9) - Synthesizing table: account
INFO: 2023-12-29 12:09:38,864 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:09:39,461 Persisting table [district]
INFO: 2023-12-29 12:09:40,224 Persisting table [client]
INFO: 2023-12-29 12:09:41,098 (6/9) - Synthesizing table: order
INFO: 2023-12-29 12:09:41,181 [SYNTHESIZER] - Start generating model samples.
INFO: 2023-12-29 12:09:41,789 